
<h2 style="text-align:center;" > Estudio de viabilidad en la delimitación de coberturas con imágenes satelitales mediante el modelo Segment Anything: Comparación con la metodología CORINE Land Cover en el municipio de Guatavita.</h2>

<h2 style="text-align:center;" >Tutorial Descarga de imagenes y procesamiento</h2>

Autores:
*   **Cristian Stiven Florez Macias**
*   **Sergio Andres Escobar Eslava** 

##  Indice
1. [INSTALACIÓN DE PAQUETES](#packages)
2. [IMPORTAR PAQUETES](#import)
3. [AUTENTICACION](#auth)
4. [ZONA DE ESTUDIO](#area)
8. [SAM](#sam)


<a name="packages"></a>

## 1. INSTALACIÓN DE PAQUETES
* **Earth Engine**: Es un paquete de python que permite la integracion con la plataforma en la nube de Google, mediante el lenguaje de programacion python permite realizar el analisis y procesamiento de las imagenes satelitales usadas en el proyecto.
* **Geemap**: Es un paquete de python de codigo abierto que hace mas interactivo el analisis geoespacial y visualizacion con earth engine, nos brinda herramientas para la visualizacion de imagenes y capas.
* **Geopandas**: Es un paquete de python de codigo abierto pensado para facilitar las operaciones con tipos geometricos (Punto, Multi-punto, Linea, Multi-linea, Poligono y Multi-poligono) mediante shapely.
* **Rasterio**: Es un paquete de python de codigo abierto que facilita la lectura y escritura de formatos para almacenar datos raster mediante una API de python que se apoya en matrices N-dimensionales y geojson.

In [ ]:
!pip install geemap geopandas rasterio

  Using cached geopandas-1.1.1-py3-none-any.whl.metadata (2.3 kB)
  Using cached rasterio-1.4.3-cp313-cp313-win_amd64.whl.metadata (9.4 kB)
  Using cached anywidget-0.9.18-py3-none-any.whl.metadata (8.9 kB)
  Using cached bqplot-0.12.45-py2.py3-none-any.whl.metadata (6.4 kB)
  Using cached eerepr-0.1.2-py3-none-any.whl.metadata (4.2 kB)
  Using cached folium-0.20.0-py2.py3-none-any.whl.metadata (4.2 kB)
  Using cached geocoder-1.38.1-py2.py3-none-any.whl.metadata (14 kB)
  Using cached ipyevents-2.0.2-py3-none-any.whl.metadata (2.9 kB)
  Using cached ipyfilechooser-0.6.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached ipyleaflet-0.20.0-py3-none-any.whl.metadata (5.3 kB)
  Using cached ipytree-0.2.2-py2.py3-none-any.whl.metadata (849 bytes)
  Using cached matplotlib-3.10.5-cp313-cp313-win_amd64.whl.metadata (11 kB)
  Using cached numpy-2.3.2-cp313-cp313-win_amd64.whl.metadata (60 kB)
  Using cached pandas-2.3.1-cp313-cp313-win_amd64.whl.metadata (19 kB)
  Using cached pyperclip-1.9.0-

<a name="import"></a>

## 2. IMPORTAR PAQUETES

Los paquetes en python son una excelente opcion para estructurar los modulos realizados en python, nos permite cargar un "módulo A.B designa un submódulo B en un paquete llamado A."(pythondocs, s.f) por ejemplo tomamos el multimodulo a:geemap con su modulo b:eefolium y le designamos un submodulo emap, esto nos da la posibilidad de no preocuparnos por nombres duplicados en nuestro proyecto.


In [ ]:
import ee
import geemap.eefolium as emap
import pandas as pd
import eeconvert as eec
import geemap
import geopandas as gpd
import rasterio as rio
import numpy as np
import matplotlib.pyplot as plt
import geojson
import os
from shapely.geometry import box
import json, requests
from rasterio.transform import from_origin
from tqdm.notebook import tqdm


<a name="import"></a>

## 3. AUTENTICACIÓN
Antes de iniciar debemos estar registrados en Google Engine y crear un nuevo proyecto.El termino de autenticacion se refiere al proceso por el cual se va a realizar el inicio de sesion mediante el cliente de python a Google Earth Engine, esta funcion usa el estandar OAuth2.0 y genera un token persistente almacenado en nuestra maquina local o el servicio de Google Collab, el metodo inicializar toma el token persistente y carga nuestro proyecto de Google Drive en la maquina local.

In [ ]:
ee.Authenticate()
ee.Initialize(project='ee-adam-segmentation')

<a name="import"></a>

# 4. ZONA DE ESTUDIO

Descripcion realizada en la monografia

In [ ]:
def generate_plot():
    

In [ ]:
ISO = 'COL' # "COL" es el codigo ISO para Colombia
ADM = 'ADM2' # Equivalente a distritos administrativos

# Busqueda de geoBoundaries mediante el servicio    
url = f"https://www.geoboundaries.org/api/current/gbOpen/{ISO}/{ADM}"
r = requests.get(url)
download_path = r.json()["gjDownloadURL"]

# Guardado del resultado en formato GeoJSON
filename = 'geoboundary.geojson'
geoboundary = requests.get(download_path).json()
with open(filename, 'w') as file:
   geojson.dump(geoboundary, file)

# Leer los datos con GeoPandas
geoboundary = gpd.read_file(filename)
geoboundary.sample(3)
shape_name = 'Guatavita'
fig, ax = plt.subplots(1, figsize=(10,10))
geoboundary[geoboundary.shapeName == shape_name].plot('shapeName', legend=True, ax=ax);

# Obtener la geometría de forma para Boyaca
regiongdf  = geoboundary.loc[geoboundary.shapeName == shape_name]
centroid = regiongdf.iloc[0].geometry.centroid.coords[0]
region = eec.gdfToFc(regiongdf) #geodataframe a feature collection